<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/Lucia/TestAndTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys


In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b Lucia https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 228 (delta 53), reused 18 (delta 8), pack-reused 137
Receiving objects: 100% (228/228), 181.98 KiB | 1.00 MiB/s, done.
Resolving deltas: 100% (129/129), done.


# Import packages

In [0]:
from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Subset, DataLoader
from torch.nn import functional as F

# Hyper-Parameters

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
BATCH_SIZE = 128
NUM_WORKERS = 100
TASK_SIZE = 10
############################################
#NUM_EPOCHS = 70
NUM_EPOCHS = 5
############################################


WEIGHT_DECAY = 0.00001
LR = 2
STEP_SIZE = [49,63]
GAMMA = 1/5

# Define Network

In [0]:
resNet = ResNet.resnet32(num_classes=100)
resNet = resNet.to(DEVICE)

In [0]:
resnet_transformer = transforms.Compose([transforms.Resize(32), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])


# Define DataSets

In [7]:
trainDS = Dataset(train=True, transform = resnet_transformer)
testDS = Dataset(train=False, transform = resnet_transformer)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


# Useful plots

The function plotEpoch plots, at the end of each task, how accuracy and loss change during the training phase. It show

*   Validation and Training Accuracy
*   Validation and Training Loss

The function plotTask, for each task, how the accuracy on the validation set change when adding new tasks


In [0]:
def plotTask(pars_tasks):
  import numpy as np
  import matplotlib.pyplot as plt

  x_tasks =  np.linspace(10, 100, 10)

  plt.plot(x_tasks, pars_tasks ,'b', label='Accuracy')
  plt.xlabel("Epoch")
  plt.title('Accuracy over classes')
  plt.legend(['Validation Accuracy'])
  plt.grid(True)
  plt.show()



# Train and evaluation phase

In [0]:
pars_tasks = []
test_indexes = []

for task in range(0, 100, TASK_SIZE):
  pars_tasks.insert(task, 0)

for task in range(0, 100, TASK_SIZE):

  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes)
  test_dataset = Subset(testDS, test_indexes)

  train_loader = DataLoader( train_dataset, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)
  test_loader = DataLoader( test_dataset, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE )

  if(task == 0):
    torch.save(resNet, 'resNet_task{0}.pt'.format(task))
  
  utils.trainfunction(task, train_loader)
  param = utils.evaluationTest(task, test_loader) #evaluate test set at step task
  pars_tasks[int(task/10)] = param #pars_task[i] = (accuracy, loss) at i-th task	

#plotTask(pars_tasks)


[2.0]     [2.0]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:396: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


class loss = 0.8002712726593018 dist loss = tensor([0.], device='cuda:0')
class loss = 0.4017544090747833 dist loss = tensor([0.], device='cuda:0')
class loss = 0.20246699452400208 dist loss = tensor([0.], device='cuda:0')
class loss = 0.14069250226020813 dist loss = tensor([0.], device='cuda:0')
class loss = 0.11152492463588715 dist loss = tensor([0.], device='cuda:0')
class loss = 0.0943206176161766 dist loss = tensor([0.], device='cuda:0')
class loss = 0.08273766934871674 dist loss = tensor([0.], device='cuda:0')
class loss = 0.07480308413505554 dist loss = tensor([0.], device='cuda:0')
class loss = 0.06932876259088516 dist loss = tensor([0.], device='cuda:0')
class loss = 0.0646679550409317 dist loss = tensor([0.], device='cuda:0')
class loss = 0.06074144318699837 dist loss = tensor([0.], device='cuda:0')
class loss = 0.05814637616276741 dist loss = tensor([0.], device='cuda:0')
class loss = 0.05543847009539604 dist loss = tensor([0.], device='cuda:0')
class loss = 0.05358245223760